#  Введение в git


Что происходит в наш с вами типичный рабочий день:

- Добавление новых фич
> Как это сделать? Можно изменять непосредственно рабочий проект, набажить, забыть, как было раньше, все испортить. Можно скопировать проект (не забыть осознанно назвать папку), добавить фичу в копии, потом мучительно "слиять", потому что кто-то уже изменил рабочий проект, а это затронуло вещи, используемые новой фичей. Каша, неразбериха.
- Фикс багов
> Копируем проект, фиксим баг, сливаем обратно. Все это с теми же проблемами.
- Развитие альтернативных версий проекта
> Копируем проект, долго разрабатываем. При слиянии еще больше кошмаров.


Все эти кейсы становятся много менее кошмарными при использовании git. В этом ноутбуке поговим о его базовой структуре и командах.



## Базовое устройство

Здесь будет описано устройство репозитория, а именно что, где и как все хранится, это будет нужно для понимания, что на самом деле делают все команды.

#### На каждой машине хранится копия всего репозитория.


#### В самом репозитории хранятся снапшоты -- "слепки" (сжатые копии) всей файловой системы в каком-то состоянии проекта.

- В каждом снапшоте хранятся копии(!) рабочей директории (а не diff-ы) 
- Если в новом снапшоте файл остался неизменным, то будет хранится ссылка на файл в предыдущем слепке.

![snapshots](https://git-scm.com/book/en/v2/images/snapshots.png)

#### Снапшоты индексируются коммитами

- Коммит - объект, содержащий ссылку на слепок, информацию об авторе коммита, комментарий, а также ссылки на предыдущий коммит (или несколько ссылок, если этот коммит -- результат слияния веток).

![commits_snapshots](https://git-scm.com/book/en/v2/images/commits-and-parents.png)


#### Ветка -- это указатель на коммит.

- На машине она представляет собой лишь файл, содержащий чек-сумму коммита
- На текущую (рабочую) ветку указывает указатель HEAD (но он может указывать и на коммит)
- Переключение между ветками пердставляет собой просто перенос указателя.

![branch](https://git-scm.com/book/en/v2/images/advance-master.png)

---

## Основные принципы

Осознанавание базовых принципов работы с файлами проекта, отслеживаемого git'ом, позволит избежать большинства ошибок и сэкономит много времени.


#### Файлы в репозитории могут находиться в трех различных состояниях: commited, modified, staged.

- Commited - файлы сохранены в локальной БД
- Modified - файлы изменены, но изменения не внесены в локальную БД
- Staged - файлы помечены, будут включены в следующий коммит



#### Соответственно, в гите есть три разных "пространства":  repository, working directory, staging area 
 
- Repository -- Хранятся метаданные и БД репозитория.
- Working directory -- локальная копия рабочей версии проекта
- Staging area (или index) -- файлы, которые войдут в следующий коммит

![states](https://git-scm.com/book/en/v2/images/areas.png)

#### Рабочий процесс


- Меняем файлы в working directory
- Указываем в staging area, какие именно изменения будут учтены при следующем коммите
- Делаем коммит (записываем снапшот в БД и создаем коммит-объект)

![](https://git-scm.com/book/en/v2/images/lifecycle.png)

---


# Базовые команды


## Создание репозитория





### Создание локального репозитория

#### git init



In [ ]:
!git init folder

Внутри folder создается папка .git -- сам репозиторий. Внутри нее содержатся коммиты, HEAD, конфиг и тп.


### Клонирование

#### git clone

git clone url folder -- клонирует репозиторий по адресу url в folder, делает чекаут активной ветки проекта.

In [ ]:
!git clone folder folder_clone

Флаги:
- -n == --no-checkout -- никуда не ставить HEAD (не делать чекаутов)
- -b == --branch name -- сделать чекаут ветки name

## Работа с репозиторием

### Работа с локальной копией







In [ ]:
import os
os.chdir('folder')

## git status

узнать текущий статус репозитория

In [ ]:
!git status

Флаги:
- -s == --short короткий вывод (о нем позднее)
- --porcelain -- вывод, удобный для парсинга скриптами
- --long -- длинный вывод (включен по дефолту) 
- -u == --untracked mode -- как показывать неотслеживаемые файлы (no - не показывать)

Сделаем видимость работы....

In [ ]:
with open('some_file.txt', 'w') as file:
    file.writelines(['work\n','hard\n'])

In [ ]:
!git status

In [ ]:
!git status -u no

In [ ]:
!git status -s

 -s: неотслеживаемые файлы обозначены ??. Левая колонка отвечает за staging area, правая -- за working tree. А == added, M == modified, D == deleted, ...

In [ ]:
!git status --porcelain

В чем разница между двумя последними флагами? Ни в чем, но вывод последней не зависит от настроек пользователя и версий git

## git add

Хотим включить some_file в следующий коммит. git советует воспользоваться add:

In [ ]:
!git add some_file.txt

#### Если хотим посмотреть на то, что произойдет после выхова команды, то нужно поставить флаг --dry-run == -n:

In [ ]:
!git add -n some_file.txt

Теперь some_file находится в состоянии staged и находится в индексе (index, staging area)

Что будет, если мы опять изменим файл?

In [ ]:
with open('some_file.txt', 'a') as file:
    file.write('work harder! \n')

In [ ]:
!git status

Обратите внимание, что в следующий коммит пойдет только первая версия файла!

In [ ]:
!git add some_file.txt

In [ ]:
!git status -s

Теперь в коммит пойдет последняя версия файла

## git commit

Теперь хотим записать изменения в репозиторий. Для этого нужно сделать коммит:

In [ ]:
!git commit -m "added some file"

#### Писать add, а потом commit иногда бессмысленно, первый этап можно пропустить:

In [ ]:
with open('some_file.txt', 'a') as file:
    file.write('i\'m tired')

In [ ]:
!git commit -a -m "modified some file"

также флаг -а удалит из индекса все файлы, которые были удалены из рабочей директории

Если хотим изменить содержание коммита предыдущего сообщения, то набираем

$ git commit --amend

откроется редактор, в котором можно будет все исправить (или ставим флаг -m "текст коммита").

Добавление этого флага также учтет все произошедшие изменения в staging area. Это способ "поправить" последний коммит, если не хочется ими "спамить" (на самом деле создастся новый коммит, который заменит предыдущий)

Для dry run нужно добавить флаг --dry-run

### git rm

Удаляет файл из индекса и рабочей директории

- -r - рекурсивное удаление
- --dry-run

In [ ]:
!git rm file.txt
!git status -s

In [ ]:
!git commit -m "deleted file"

In [ ]:
!dir

#### Если мы сделали файл staged, а потом передумали, то можно вызвать rm с флагом --cached (удалит из индекса, но не с машины)

In [ ]:
with open('file.txt', 'a') as file:
    file.write('i\'m so useless....')
    
!git add file.txt
!git status -s

In [ ]:
!git rm --cached file.txt

In [ ]:
!git status -s


#### Что будет, если просто удалить файл из working tree:

In [ ]:
!git add file.txt
!git commit -m "added file"

In [ ]:
!del file.txt

In [ ]:
!git status

In [ ]:
!git commit -a -m "deleted file"

In [ ]:
!git status

In [ ]:
!dir

## просмотр изменений

## git diff

In [ ]:
!dir

In [ ]:
with open('some_file.txt','a') as file:
    file.write('lalala\n')

In [ ]:
!git diff

In [ ]:
!git add some_file.txt

In [ ]:
!git diff

#### чтобы посмотреть разницу между файлами в последнем коммите и staged, нужно передать флаг cached (или staged):

In [ ]:
!git diff --staged

#### посмотреть разность staged с другой веткой (подставь вместо HEAD имя ветки):

In [ ]:
!git diff HEAD